In [1]:
# ============================================
# Phase 1: Emotion Chatbot – Multi-label Training
# ============================================

import os
import json
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import RobertaTokenizer, TrainingArguments, Trainer, EarlyStoppingCallback
from transformers import RobertaPreTrainedModel, RobertaModel, RobertaConfig
import torch.nn as nn
import datasets

# --------------------------
# 1) Config & Device
# --------------------------
OUTPUT_DIR = "../results/models/phase1_emotion_roberta"
LOG_DIR = "../results/logs"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --------------------------
# 2) Load dataset
# --------------------------
df = pd.read_csv("../data/dataset2_emotions/processed/emotions_clean.csv")
print("Raw dataset shape:", df.shape)
print(df.head())

# 👇 If dataset has "emotion" column (single string label), convert to one-hot
if "emotion" in df.columns:
    all_emotions = sorted(df["emotion"].unique().tolist())
    print("Unique emotions:", all_emotions)

    # create one-hot columns
    for emo in all_emotions:
        df[emo] = (df["emotion"] == emo).astype(int)

    df = df.drop(columns=["emotion"])

    # group by text → merge duplicate texts with multiple labels
    df = df.groupby("text")[all_emotions].max().reset_index()

print("Processed dataset shape:", df.shape)
print(df.head())

emotion_cols = [c for c in df.columns if c != "text"]
num_labels = len(emotion_cols)
print("Emotion labels:", emotion_cols)

# --------------------------
# 3) Train/Val/Test split
# --------------------------
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# --------------------------
# 4) Tokenizer & Dataset
# --------------------------
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_and_format(df):
    encodings = tokenizer(
        df["text"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=128
    )
    labels = df[emotion_cols].astype(float).values.tolist()

    dataset = datasets.Dataset.from_dict({
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"],
        "labels": labels
    })
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return dataset

train_ds = tokenize_and_format(train_df)
val_ds   = tokenize_and_format(val_df)
test_ds  = tokenize_and_format(test_df)

# --------------------------
# 5) Custom Roberta Model (multi-label with BCEWithLogitsLoss)
# --------------------------
class RobertaForMultiLabel(RobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.post_init()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0, :]  # CLS token
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels.float())
        return {"loss": loss, "logits": logits}

config = RobertaConfig.from_pretrained("roberta-base", num_labels=num_labels)
model = RobertaForMultiLabel.from_pretrained("roberta-base", config=config).to(device)

# --------------------------
# 6) Metrics
# --------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int()
    labels = torch.tensor(labels)

    acc = accuracy_score(labels, preds)
    micro_f1 = f1_score(labels, preds, average="micro", zero_division=0)
    macro_f1 = f1_score(labels, preds, average="macro", zero_division=0)

    return {"accuracy": acc, "micro_f1": micro_f1, "macro_f1": macro_f1}

# --------------------------
# 7) Training setup with Early Stopping
# --------------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,   # increase if GPU memory allows
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    num_train_epochs=3,               # start small, can increase later
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
    greater_is_better=True,
    logging_dir=LOG_DIR,
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=2,
    fp16=True,                        # mixed precision (fast on GPU)
    dataloader_num_workers=2
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# --------------------------
# 8) Train with Early Stopping
# --------------------------
trainer.train()

# --------------------------
# 9) Evaluate on test set
# --------------------------
results = trainer.evaluate(test_ds)
print("Test results:", results)

# --------------------------
# 9.1) Find best thresholds on validation set
# --------------------------
import numpy as np
from sklearn.metrics import f1_score

# Get validation predictions
val_outputs = trainer.predict(val_ds)
val_logits = val_outputs.predictions
val_labels = np.array(val_outputs.label_ids)

val_probs = torch.sigmoid(torch.tensor(val_logits)).numpy()

def find_best_thresholds(y_true, y_pred_probs, emotion_cols):
    thresholds = {}
    for i in range(y_true.shape[1]):
        best_t, best_f1 = 0.5, 0.0
        for t in np.arange(0.1, 0.9, 0.05):
            preds = (y_pred_probs[:, i] >= t).astype(int)
            f1 = f1_score(y_true[:, i], preds, zero_division=0)
            if f1 > best_f1:
                best_t, best_f1 = t, f1
        thresholds[emotion_cols[i]] = best_t
    return thresholds

thresholds = find_best_thresholds(val_labels, val_probs, emotion_cols)
print("\nBest thresholds per emotion:")
print(thresholds)

# --------------------------
# 10) Predict on few examples using thresholds
# --------------------------
def predict_emotions(prob, emotion_cols, thresholds, max_labels=5, neutral_label="neutral"):
    preds = []

    # Step 1: Apply thresholds
    for i, p in enumerate(prob):
        if p >= thresholds.get(emotion_cols[i], 0.5):
            preds.append((emotion_cols[i], p))

    # Step 2: Fallback if nothing passes threshold
    if not preds:
        top_idx = np.argmax(prob)
        preds = [(emotion_cols[top_idx], prob[top_idx])]

    # Step 3: Sort by probability (high → low)
    preds = sorted(preds, key=lambda x: x[1], reverse=True)

    # Step 4: Keep at most `max_labels`
    preds = preds[:max_labels]

    # Step 5: Drop "neutral" if other strong emotions exist
    labels = [lbl for lbl, p in preds]
    if neutral_label in labels and len(labels) > 1:
        labels = [lbl for lbl in labels if lbl != neutral_label]

    return labels


sample_texts = [
    "I am really anxious and can't sleep 😟",
    "I feel so happy and grateful today 😊",
    "I am angry but also disappointed 😡😞",

    "I feel like nothing matters anymore.",
    "Life has been so heavy lately.",
    "I can’t stop crying, everything hurts.",

    "Why does this always happen to me?!",
    "I’m so pissed off right now.",
    "People never listen and it makes me furious.",

    "I’m scared I’ll fail my exams.",
    "What if something bad happens tomorrow?",
    "I can’t stop worrying, my heart is racing.",

    "I finally got the job I wanted!",
    "I feel so grateful today.",
    "This is the happiest I’ve been in years.",

    "I really care about you.",
    "Spending time with my family makes me feel loved.",
    "I think I’m falling for them.",

    "Wow, I didn’t see that coming at all!",
    "You won’t believe what just happened.",
    "That was totally unexpected.",

    "I’m nervous about the interview, but also excited.",  # (fear + joy)
    "I’m happy for them, but I feel lonely inside.",       # (joy + sadness)
    "I’m so angry, but also deeply hurt.",                 # (anger + sadness)
    "I feel grateful but anxious about the future."        # (joy + fear)
]
encodings = tokenizer(sample_texts, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
outputs = model(**encodings)
probs = torch.sigmoid(outputs["logits"]).detach().cpu().numpy()

for text, prob in zip(sample_texts, probs):
    preds = predict_emotions(prob, emotion_cols, thresholds, max_labels=5, neutral_label="neutral")
    print(f"\nText: {text}\nPredicted emotions: {preds}")

# ✅ Save final model + tokenizer (Phase 1 emotions)
save_dir = "../results/models/phase1_emotion_roberta"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"Model saved at {save_dir}")

# ==========================
# Save idx2emotion.json
# ==========================
idx2emotion = {i: emo for i, emo in enumerate(emotion_cols)}
with open(os.path.join(save_dir, "idx2emotion.json"), "w") as f:
    json.dump(idx2emotion, f, indent=4)

print(f"idx2emotion.json saved at {save_dir}")

# ==========================
# Save thresholds.json
# ==========================
with open(os.path.join(save_dir, "thresholds.json"), "w") as f:
    json.dump(thresholds, f, indent=4)

print(f"thresholds.json saved at {save_dir}")



c:\Users\rijju\Desktop\mental-health-chatbot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Raw dataset shape: (269529, 2)
                                                text  emotion
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
2   im grabbing a minute to post i feel greedy wrong    anger
3  i am ever feeling nostalgic about the fireplac...     love
4                               i am feeling grouchy    anger
Unique emotions: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']
Processed dataset shape: (77678, 29)
                                                text  admiration  amusement  \
0   "If you don't wear BROWN AND ORANGE...YOU DON...           0          0   
1   "What do Scottish people loo

c:\Users\rijju\Desktop\mental-health-chatbot\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForMultiLabel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 100/11652 [01:25<1:38:56,  1.95it/s]

{'loss': 0.3267, 'grad_norm': 0.529172420501709, 'learning_rate': 1.982835564709921e-05, 'epoch': 0.03}


  2%|▏         | 200/11652 [02:19<1:48:43,  1.76it/s]

...

{'train_runtime': 5052.0586, 'train_samples_per_second': 36.901, 'train_steps_per_second': 2.306, 'train_loss': 0.15191054880271201, 'epoch': 3.0}


100%|██████████| 486/486 [00:45<00:00, 10.74it/s]


Test results: {'eval_loss': 0.14884571731090546, 'eval_accuracy': 0.3532440782698249, 'eval_micro_f1': 0.6067224774973249, 'eval_macro_f1': 0.5014063659130918, 'eval_runtime': 88.3739, 'eval_samples_per_second': 87.899, 'eval_steps_per_second': 5.499, 'epoch': 3.0}


100%|██████████| 486/486 [00:44<00:00, 10.81it/s]



Best thresholds per emotion:
{'admiration': np.float64(0.3500000000000001), 'amusement': np.float64(0.5000000000000001), 'anger': np.float64(0.40000000000000013), 'annoyance': np.float64(0.25000000000000006), 'approval': np.float64(0.20000000000000004), 'caring': np.float64(0.30000000000000004), 'confusion': np.float64(0.30000000000000004), 'curiosity': np.float64(0.30000000000000004), 'desire': np.float64(0.45000000000000007), 'disappointment': np.float64(0.20000000000000004), 'disapproval': np.float64(0.25000000000000006), 'disgust': np.float64(0.20000000000000004), 'embarrassment': np.float64(0.30000000000000004), 'excitement': np.float64(0.25000000000000006), 'fear': np.float64(0.45000000000000007), 'gratitude': np.float64(0.40000000000000013), 'grief': np.float64(0.15000000000000002), 'joy': np.float64(0.40000000000000013), 'love': np.float64(0.3500000000000001), 'nervousness': np.float64(0.15000000000000002), 'neutral': np.float64(0.30000000000000004), 'optimism': np.float64(0.3

In [2]:
# ✅ Save final model + tokenizer (Phase 1 emotions)
save_dir = "../results/models/phase1_emotion_roberta"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"Model saved at {save_dir}")


Model saved at ../results/models/phase1_emotion_roberta


In [3]:
# --------------------------
# 9.1) Find best thresholds on validation set
# --------------------------
import numpy as np
from sklearn.metrics import f1_score

# Get validation predictions
val_outputs = trainer.predict(val_ds)
val_logits = val_outputs.predictions
val_labels = np.array(val_outputs.label_ids)

val_probs = torch.sigmoid(torch.tensor(val_logits)).numpy()

def find_best_thresholds(y_true, y_pred_probs, emotion_cols):
    thresholds = {}
    for i in range(y_true.shape[1]):
        best_t, best_f1 = 0.5, 0.0
        for t in np.arange(0.1, 0.9, 0.05):
            preds = (y_pred_probs[:, i] >= t).astype(int)
            f1 = f1_score(y_true[:, i], preds, zero_division=0)
            if f1 > best_f1:
                best_t, best_f1 = t, f1
        thresholds[emotion_cols[i]] = best_t
    return thresholds

thresholds = find_best_thresholds(val_labels, val_probs, emotion_cols)
print("\nBest thresholds per emotion:")
print(thresholds)

# --------------------------
# 10) Predict on few examples using thresholds
# --------------------------
def predict_emotions(prob, emotion_cols, thresholds, max_labels=5, neutral_label="neutral"):
    preds = []

    # Step 1: Apply thresholds
    for i, p in enumerate(prob):
        if p >= thresholds.get(emotion_cols[i], 0.5):
            preds.append((emotion_cols[i], p))

    # Step 2: Fallback if nothing passes threshold
    if not preds:
        top_idx = np.argmax(prob)
        preds = [(emotion_cols[top_idx], prob[top_idx])]

    # Step 3: Sort by probability (high → low)
    preds = sorted(preds, key=lambda x: x[1], reverse=True)

    # Step 4: Keep at most `max_labels`
    preds = preds[:max_labels]

    # Step 5: Drop "neutral" if other strong emotions exist
    labels = [lbl for lbl, p in preds]
    if neutral_label in labels and len(labels) > 1:
        labels = [lbl for lbl in labels if lbl != neutral_label]

    return labels


sample_texts = [
    "I am really anxious and can't sleep 😟",
    "I feel so happy and grateful today 😊",
    "I am angry but also disappointed 😡😞",

    "I feel like nothing matters anymore.",
    "Life has been so heavy lately.",
    "I can’t stop crying, everything hurts.",

    "Why does this always happen to me?!",
    "I’m so pissed off right now.",
    "People never listen and it makes me furious.",

    "I’m scared I’ll fail my exams.",
    "What if something bad happens tomorrow?",
    "I can’t stop worrying, my heart is racing.",

    "I finally got the job I wanted!",
    "I feel so grateful today.",
    "This is the happiest I’ve been in years.",

    "I really care about you.",
    "Spending time with my family makes me feel loved.",
    "I think I’m falling for them.",

    "Wow, I didn’t see that coming at all!",
    "You won’t believe what just happened.",
    "That was totally unexpected.",

    "I’m nervous about the interview, but also excited.",  # (fear + joy)
    "I’m happy for them, but I feel lonely inside.",       # (joy + sadness)
    "I’m so angry, but also deeply hurt.",                 # (anger + sadness)
    "I feel grateful but anxious about the future."        # (joy + fear)
]
encodings = tokenizer(sample_texts, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
outputs = model(**encodings)
probs = torch.sigmoid(outputs["logits"]).detach().cpu().numpy()

for text, prob in zip(sample_texts, probs):
    preds = predict_emotions(prob, emotion_cols, thresholds, max_labels=5, neutral_label="neutral")
    print(f"\nText: {text}\nPredicted emotions: {preds}")

100%|██████████| 486/486 [00:47<00:00, 10.28it/s]



Best thresholds per emotion:
{'admiration': np.float64(0.3500000000000001), 'amusement': np.float64(0.5000000000000001), 'anger': np.float64(0.40000000000000013), 'annoyance': np.float64(0.25000000000000006), 'approval': np.float64(0.20000000000000004), 'caring': np.float64(0.30000000000000004), 'confusion': np.float64(0.30000000000000004), 'curiosity': np.float64(0.30000000000000004), 'desire': np.float64(0.45000000000000007), 'disappointment': np.float64(0.20000000000000004), 'disapproval': np.float64(0.25000000000000006), 'disgust': np.float64(0.20000000000000004), 'embarrassment': np.float64(0.30000000000000004), 'excitement': np.float64(0.25000000000000006), 'fear': np.float64(0.45000000000000007), 'gratitude': np.float64(0.40000000000000013), 'grief': np.float64(0.15000000000000002), 'joy': np.float64(0.40000000000000013), 'love': np.float64(0.3500000000000001), 'nervousness': np.float64(0.15000000000000002), 'neutral': np.float64(0.30000000000000004), 'optimism': np.float64(0.3

In [4]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA version (PyTorch):", torch.version.cuda)
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


CUDA available: True
CUDA version (PyTorch): 12.4
Device name: NVIDIA GeForce RTX 3050 Laptop GPU


In [5]:
import transformers, os
print(transformers.__file__)


c:\Users\rijju\Desktop\mental-health-chatbot\myenv\Lib\site-packages\transformers\__init__.py
